In [1]:
import random
import itertools
import re
import pandas as pd
from transformers import *
import torch

/home/msc2/Jupyter/jupyterenv/lib/python3.8/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/home/msc2/Jupyter/jupyterenv/lib/python3.8/site-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
2023-04-17 10:05:08.351287: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-17 10:05:08.351362: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
train_df = pd.read_csv('./train.csv')

In [5]:
tmp_df = train_df.head()

In [ ]:
# model = BertModel.from_pretrained('/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/bert-trained/', output_attentions = True)
# tokenizer = BertTokenizer.from_pretrained('/media/msc2/A6982E8F982E5DD5/SEM3/Project/Models/bert-trained/')

In [7]:
def get_indices(sents, samples):
    indices = []
    for sample in samples:
        indices.append(sents.index(sample))
    return indices

def set_indices(sents, samples, indices):
    for idx in range(len(samples)):
        sents[indices[idx]] = samples[idx]
    return sents

def generate_perms_summary(summ, num_shuffles = 10):
    '''for summaries'''
    sentences = set()
    sents = re.split('।', summ)
    if len(sents)>1:
        sents = sents[:-1]
    for _ in range(num_shuffles):
        tmp_sents = []
        for sent in sents:
            words = re.split('\s+', sent)
            tmp_sents.append(' '.join(random.sample(words, len(words))))
        final_sents = ' । '.join(tmp_sents)+' । '     
        sentences.add(final_sents)
    return sentences

def generate_perms_text(text, num_shuffles = 10, sent_frac = 0.6):
    '''for multiline sentences'''
    sentences = set()
    sents = re.split('।', text)
    if len(sents) > 1:
        sents = sents[:-1]
    for _ in range(num_shuffles):
        tmp_sample_sent_list = []
        sample_sents = random.sample(sents, int(sent_frac * (len(sents))))
        sample_indices = get_indices(sents, sample_sents)
        for sent in sample_sents:
            words = re.split('\s+', sent)
            tmp_sample_sent_list.append(' '.join(random.sample(words, len(words))))
        tmp_sent_list = set_indices(sents, tmp_sample_sent_list, sample_indices)
        sentences.add(' । '.join(tmp_sent_list))
    return sentences

In [8]:
def stats(df, label = None):
    print(f'{label} : {df.shape}')
    for col in df.columns:
        print(f'Null in {col}: {df[col].isna().sum()}')
        print(f'Duplicates in {col}: {df.duplicated(subset = [col]).sum()}')
    return

def is_dup(col):
    flag = False
    l = []
    for idx in range(len(col)-1):
        if col.loc[idx] == col.loc[idx+1]:
            l.append(idx)
            flag = True
    return flag, l

In [9]:
def apply_perms(row, num_shuffles, sent_frac = 0.6):
    perms = list(generate_perms_text(row['text'], num_shuffles = num_shuffles, sent_frac = sent_frac))
    labels = list(generate_perms_summary(row['summary'], num_shuffles = num_shuffles))
    return pd.DataFrame(list(zip(perms, labels)),columns =['text', 'summary'])

In [10]:
def generate_subset(df, percent = 0.2, num_shuffles = 3, sent_frac = 0.6):
    subset = df.sample(frac = percent)
    permutations_df = pd.DataFrame(columns = ['text', 'summary'])
    for row in subset.iterrows():
        tmp_df = apply_perms(row[1], num_shuffles, sent_frac = sent_frac)
        permutations_df = pd.concat([permutations_df, tmp_df], ignore_index = True)
    aug_df = pd.concat([df, permutations_df], ignore_index = True).sample(frac = 1, ignore_index = True)
    return aug_df

In [13]:
aug_tmp_df = generate_subset(tmp_df, percent = 1, num_shuffles = 1, sent_frac = 0.6)

In [14]:
aug_tmp_df

,text,summary
0,तेहान् गते सतम्बरमासे इति संसदीयसत्येशोधनासमि...,बालेषु विषानिलाक्रमणम् ।
1,"• डा० विडि हेगडे, मैसूरु पाकिस्ताने कुटिला राज...",इमरान् खानं कुटिला राजनीति वरयते ।
2,"• डा० विडि हेगडे, मैसूरु पाकिस्ताने कुटिला राज...",कुटिला राजनीति इमरान् खानं वरयते
3,अतिरूक्षस्य प्रलयदुरन्तस्य पश्चात् केरलानां प्...,केरले राज्यस्तरीयः कायिकोत्सवः समारब्धः।
4,तेहान् गते सतम्बरमासे इति संसदीयसत्येशोधनासमि...,बालेषु विषानिलाक्रमणम्
5,सूरतविमानस्थानकात् राजस्थानं प्रति गन्तुकामाना...,फरवरीमासात् सूरतनगरात् अतिरिक्तविमानसेवा स्थिर...
6,"- ७०,३१,३७७ अभवन् जनाः विश्वे कोविड् अद्यावधि ...","सप्ततिलक्षं कोविड्रोगिणः मरणानि आविश्वं , चतुर..."
7,सूरतविमानस्थानकात् राजस्थानं प्रति गन्तुकामाना...,स्थिरीकृता अतिरिक्तविमानसेवा सूरतनगरात् फरवरीम...
8,प्रथमः सार्वजनीनसङ्गमः प्रलयदुरन्तस्य केरलानां...,राज्यस्तरीयः समारब्धः कायिकोत्सवः केरले ।
9,"विश्वे अद्यावधि ७०,३१,३७७ जनाः कोविड् - १९रोगब...","आविश्वं कोविड्रोगिणः सप्ततिलक्षं , मरणानि चतुर..."


In [17]:
before_aug = aug_tmp_df['text'][0]

In [18]:
after_aug = aug_tmp_df['text'][4]

In [19]:
before_aug

'तेहान्  गते सतम्बरमासे इति संसदीयसत्येशोधनासमितेः बस्वसंहिता उल्लंघितेति बद्धा सती सदस्यः महम्मद हसन् असफारि  । प्राकटयत्   ।  पचसहस्रं छात्राः बाधिताः विषानिलप्रयोगेण ।  शालावधी एव एवं सुगुप्तरीत्या कृतकार्येण छात्राः कारागृहे एव महसा अमिनि मृता । समजनयत् आदेशं इरानुप्रान्ते प्रकाशिता प्रकरणं प्राणापाय जाता वार्ता  हिजाविरोधिप्रतिभटना आतङ्ककारिणी इदानीं इति तत् श्वासोच्छ्वाससमस्या, ।  महिला बहिरंगतया शिरोभ्रमणं, इत्यादिभिः पीडिताः । २५% एतेषु बुर्खादहनं  कहर इस्लाम् चिकित्सां गत्वा प्रशासनाय सम्प्रदायस्य वैद्यालयान् मनं, हिजाब, कृत्वा छत्राः प्राप्नुवन् ।  पथ्याह्वानं प्रादुः । व्यक्तीकृतम् तनेपथ्ये  विश्वस्तरे खण्डनं विद्यालयेषु विश्वस्तरे एतद्विषये प्रतिभटनाकार्यभूतेषु । प्रति विद्यालयानाम् खोमेनिः उपरि एतादृशविरोधिछात्रान् कृतं वर्तते आयातुल्ला स्वीकरणीयाः असूचयत् इति उग्रक्रमाः २३०  अपसाधिनः धार्मिकनायकः निग्रहीतुं विषानिलस्य आक्रमणं'

In [20]:
after_aug

'तेहान्  गते सतम्बरमासे इति संसदीयसत्येशोधनासमितेः बस्वसंहिता उल्लंघितेति बद्धा सती सदस्यः महम्मद हसन् असफारि । प्राकटयत् । विषानिलप्रयोगेण पचसहस्रं छात्राः बाधिताः। शालावधी एव एवं सुगुप्तरीत्या कृतकार्येण छात्राः कारागृहे एव महसा अमिनि मृता। तत् प्रकरणं इदानीं इरानुप्रान्ते प्राणापाय समजनयत् इति आतङ्ककारिणी वार्ता प्रकाशिता आदेशं हिजाविरोधिप्रतिभटना श्वासोच्छ्वाससमस्या, जाता। महिला बहिरंगतया शिरोभ्रमणं, इत्यादिभिः पीडिताः। हिजाब, बुर्खादहनं कृत्वा कहर एतेषु २५% छत्राः वैद्यालयान् इस्लाम् सम्प्रदायस्य प्रशासनाय गत्वा मनं, चिकित्सां प्राप्नुवन्। पथ्याह्वानं प्रादुः। तनेपथ्ये एतद्विषये विश्वस्तरे विश्वस्तरे खण्डनं प्रतिभटनाकार्यभूतेषु विद्यालयेषु व्यक्तीकृतम्। अपसाधिनः प्रति एतादृशविरोधिछात्रान् निग्रहीतुं उग्रक्रमाः स्वीकरणीयाः इति २३० विद्यालयानाम् उपरि धार्मिकनायकः आयातुल्ला खोमेनिः विषानिलस्य आक्रमणं कृतं वर्तते असूचयत्।'

In [512]:
aug_df = generate_subset(train_df, 0.5, 1, 0.6)

In [514]:
stats(aug_df)

None : (15779, 2)
Null in text: 0
Duplicates in text: 652
Null in summary: 0
Duplicates in summary: 0


In [528]:
len(is_dup(aug_df['text'])[1])

0

In [527]:
aug_df.shape

(15779, 2)

In [526]:
aug_df.to_csv('./augmented_train_data.csv', index = False)

In [536]:
t_df = pd.read_csv('./augmented_train_data.csv')
stats(t_df)

None : (15779, 2)
Null in text: 0
Duplicates in text: 652
Null in summary: 0
Duplicates in summary: 0


In [537]:
val_df = pd.read_csv('./val.csv')
stats(val_df)

None : (1500, 2)
Null in text: 0
Duplicates in text: 0
Null in summary: 0
Duplicates in summary: 0


In [538]:
test_df = pd.read_csv('./test.csv')
stats(test_df)

None : (245, 2)
Null in text: 0
Duplicates in text: 0
Null in summary: 0
Duplicates in summary: 0


In [543]:
import nltk